In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
#loading channels sheet
from os import listdir
filepaths=["./channel/day=25/hour=17/"+f for f in listdir("./channel/day=25/hour=17/") if f.endswith('.csv')]
rawchan=[]
for filename in filepaths:
    temp=pd.read_csv(filename, header=0, sep="|")
    rawchan.append(temp)
df_channel=pd.concat(rawchan, axis=0, ignore_index=True)

In [3]:
#loading video sheets
rawvideo=[]
filepaths=["./video/day=25/hour=17/"+f for f in listdir("./video/day=25/hour=17/") if f.endswith(".csv")]
for filename in filepaths:
    temp=pd.read_csv(filename, header=0, sep="|", error_bad_lines=False)
    rawvideo.append(temp)
df_video=pd.concat(rawvideo, axis=0, ignore_index=True)

b'Skipping line 294: expected 16 fields, saw 17\n'
b'Skipping line 18: expected 16 fields, saw 17\n'
b'Skipping line 191: expected 16 fields, saw 17\n'
b'Skipping line 86: expected 16 fields, saw 17\n'
b'Skipping line 206: expected 16 fields, saw 17\n'
b'Skipping line 23: expected 16 fields, saw 17\n'


In [4]:
#remove commentCount and rename channelId
df_channel=df_channel.drop(["commentCount"], axis=1)
df_channel=df_channel.rename(columns={"id":"channelId"})

In [5]:
#date and time conversion
df_channel["collectedTime"]=pd.to_datetime(df_channel.collectedTime)
df_channel["publishedAt"]=pd.to_datetime(df_channel.publishedAt)

In [6]:
#rename videoId
df_video=df_video.rename(columns={"id":"videoId"})

In [7]:
#date and time conversion
df_video["collectedTime"]=pd.to_datetime(df_video.collectedTime)
df_video["publishedAt"]=pd.to_datetime(df_video.publishedAt)

In [8]:
#title and category from channel to video
chtovid=df_channel[["channelId", "title", "category", "sub_category"]]#creating sub df#creating sub df
chtovid=chtovid.rename(columns={"channelId":"channelId", "title":"ch_title", "category":"ch_category", "sub_category":"ch_sub_category"})
df_video=pd.merge(df_video, chtovid, on="channelId")#merge above title and category to video

In [9]:
#parse date out of publishedAt from df_video
df_video["pubDate"]=df_video.publishedAt.astype(str).str[:-15]
df_video["pubDate"]=pd.to_datetime(df_video.pubDate)

In [10]:
#parse date out of publishedAt from df_channel
df_channel["pubDate"]=df_channel.publishedAt.astype(str).str[:-15]
df_channel["pubDate"]=pd.to_datetime(df_channel.pubDate)

In [11]:
#parse out collection date from df_channel
df_channel["colDate"]=df_channel.collectedTime.astype(str).str[:-15]
df_channel["colDate"]=pd.to_datetime(df_channel.colDate)

In [12]:
#date since the channel was created
df_channel["circa_D"] = (df_channel.colDate - df_channel.pubDate).dt.days

In [13]:
#latest periodical data

#get the last 1-day data
df1D=df_video.set_index("pubDate").sort_index().last("1D")
df1D=df1D.reset_index()
#get the last 1-week data
df1W=df_video.set_index("pubDate").sort_index().last("1W")
df1W=df1W.reset_index()
#get the last 1-month data
df1M=df_video.set_index("pubDate").sort_index().last("1M")
df1M=df1M.reset_index()
#get the last 3-month data
df3M=df_video.set_index("pubDate").sort_index().last("3M")
df3M=df3M.reset_index()
#get the last 6-month data
df6M=df_video.set_index("pubDate").sort_index().last("6M")
df6M=df6M.reset_index()
#get the last 1-year data
df1Y=df_video.set_index("pubDate").sort_index().last("1Y")
df1Y=df1Y.reset_index()

In [14]:
#extract numbers of vids per latest periods

#1D
vidcount=df1D.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_1D")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_1D"]=df_channel["videoCount_1D"].fillna(value=0)#set NAs into 0
#1W
vidcount=df1W.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_1W")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_1W"]=df_channel["videoCount_1W"].fillna(value=0)#set NAs into 0
#1M
vidcount=df1M.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_1M")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_1M"]=df_channel["videoCount_1M"].fillna(value=0)#set NAs into 0
#3M
vidcount=df3M.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_3M")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_3M"]=df_channel["videoCount_3M"].fillna(value=0)#set NAs into 0
#6M
vidcount=df6M.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_6M")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_6M"]=df_channel["videoCount_6M"].fillna(value=0)#set NAs into 0
#1Y
vidcount=df1Y.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_1Y")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_1Y"]=df_channel["videoCount_1Y"].fillna(value=0)#set NAs into 0


In [15]:
#extract commentCount, like, dislike, view for latest periods
#total
like=df_video.groupby(["channelId"]).sum()#get sum
like=like.reset_index()
# like=like.drop(["viewCount", "commentCount"], axis=1).reset_index()#drop pointless columns and reset index
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#1D
df1D=df1D.rename(columns={"viewCount":"viewCount_1D", "likeCount":"likeCount_1D", "dislikeCount":"dislikeCount_1D" ,"commentCount":"commentCount_1D"})
like=df1D.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#1W
df1W=df1W.rename(columns={"viewCount":"viewCount_1W", "likeCount":"likeCount_1W", "dislikeCount":"dislikeCount_1W", "commentCount":"commentCount_1W"})
like=df1W.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#1M
df1M=df1M.rename(columns={"viewCount":"viewCount_1M", "likeCount":"likeCount_1M", "dislikeCount":"dislikeCount_1M", "commentCount":"commentCount_1M"})
like=df1M.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#3M
df3M=df3M.rename(columns={"viewCount":"viewCount_3M", "likeCount":"likeCount_3M", "dislikeCount":"dislikeCount_3M", "commentCount":"commentCount_3M"})
like=df3M.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#6M
df6M=df6M.rename(columns={"viewCount":"viewCount_6M", "likeCount":"likeCount_6M", "dislikeCount":"dislikeCount_6M", "commentCount":"commentCount_6M"})
like=df6M.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#1Y
df1Y=df1Y.rename(columns={"viewCount":"viewCount_1Y", "likeCount":"likeCount_1Y", "dislikeCount":"dislikeCount_1Y","commentCount":"commentCount_1Y"})
like=df1Y.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel


In [16]:
#Now calculating first periods' variables

In [17]:
#get first publication date for each channel from df_video
firstvid=df_video.sort_values(["channelId", "pubDate"]).groupby(["channelId"]).first().reset_index()
firstvid["dayone"]=firstvid["pubDate"]#avoiding naming conflicts

In [18]:
#change firstvid into viddate and add term periods
viddate=firstvid[["channelId", "dayone"]]
viddate
viddate["f1W"]=viddate.dayone+pd.DateOffset(weeks=1)
viddate["f1M"]=viddate.dayone+pd.DateOffset(months=1)
viddate["f3M"]=viddate.dayone+pd.DateOffset(months=3)
viddate["f6M"]=viddate.dayone+pd.DateOffset(months=6)
viddate["f1Y"]=viddate.dayone+pd.DateOffset(years=1)
viddate["f2Y"]=viddate.dayone+pd.DateOffset(years=2)
viddate["f3Y"]=viddate.dayone+pd.DateOffset(years=3)
viddate["f4Y"]=viddate.dayone+pd.DateOffset(years=4)
viddate["f5Y"]=viddate.dayone+pd.DateOffset(years=5)
viddate#created term periods for each channel circa their first pubDate

,channelId,dayone,f1W,f1M,f3M,f6M,f1Y,f2Y,f3Y,f4Y,f5Y
0,UC-8R1d-KQrZy_7-EvE_w9Nw,2019-10-30,2019-11-06,2019-11-30,2020-01-30,2020-04-30,2020-10-30,2021-10-30,2022-10-30,2023-10-30,2024-10-30
1,UC-9nTjw_qHCA693UA3jyDWw,2017-03-13,2017-03-20,2017-04-13,2017-06-13,2017-09-13,2018-03-13,2019-03-13,2020-03-13,2021-03-13,2022-03-13
2,UC-9oz6OofrZnaOUQ48-LavA,2014-10-15,2014-10-22,2014-11-15,2015-01-15,2015-04-15,2015-10-15,2016-10-15,2017-10-15,2018-10-15,2019-10-15
3,UC-AuHoBXFe2e6YQ3fcA2zFA,2015-04-23,2015-04-30,2015-05-23,2015-07-23,2015-10-23,2016-04-23,2017-04-23,2018-04-23,2019-04-23,2020-04-23
4,UC-BqPABOl7c7rR5EoJJZ0UA,2016-10-11,2016-10-18,2016-11-11,2017-01-11,2017-04-11,2017-10-11,2018-10-11,2019-10-11,2020-10-11,2021-10-11
...,...,...,...,...,...,...,...,...,...,...,...
976,UCzXNBo1PYNsqSrN19XK3hPw,2017-10-09,2017-10-16,2017-11-09,2018-01-09,2018-04-09,2018-10-09,2019-10-09,2020-10-09,2021-10-09,2022-10-09
977,UCzacSE_ExLj6L5rXx2CakjQ,2016-07-04,2016-07-11,2016-08-04,2016-10-04,2017-01-04,2017-07-04,2018-07-04,2019-07-04,2020-07-04,2021-07-04
978,UCzdPXIkTEXBkbx9cQ9aYDJA,2016-01-17,2016-01-24,2016-02-17,2016-04-17,2016-07-17,2017-01-17,2018-01-17,2019-01-17,2020-01-17,2021-01-17
979,UCzmGNr7bHavW8SUg1n4KIMQ,2016-02-24,2016-03-02,2016-03-24,2016-05-24,2016-08-24,2017-02-24,2018-02-24,2019-02-24,2020-02-24,2021-02-24


In [19]:
#add term periods to df_video
df_video=pd.merge(df_video, viddate, on="channelId", how="left")

In [20]:
#create dfs with each time period
df_f1W=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f1W)]
df_f1M=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f1M)]
df_f3M=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f3M)]
df_f6M=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f6M)]
df_f1Y=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f1Y)]
df_f2Y=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f2Y)]
df_f3Y=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f3Y)]
df_f4Y=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f4Y)]
df_f5Y=df_video[(df_video.pubDate>=df_video.dayone) & (df_video.pubDate<=df_video.f5Y)]

In [21]:
#extract numbers of vids for first periods

#f1W
vidcount=df_f1W.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f1W")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f1W"]=df_channel["videoCount_f1W"].fillna(value=0)#set NAs into 0
#f1M
vidcount=df_f1M.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f1M")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f1M"]=df_channel["videoCount_f1M"].fillna(value=0)#set NAs into 0
#f3M
vidcount=df_f3M.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f3M")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f3M"]=df_channel["videoCount_f3M"].fillna(value=0)#set NAs into 0
#f6M
vidcount=df_f6M.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f6M")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f6M"]=df_channel["videoCount_f6M"].fillna(value=0)#set NAs into 0
#f1Y
vidcount=df_f1Y.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f1Y")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f1Y"]=df_channel["videoCount_f1Y"].fillna(value=0)#set NAs into 0
#f2Y
vidcount=df_f2Y.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f2Y")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f2Y"]=df_channel["videoCount_f2Y"].fillna(value=0)#set NAs into 0
#f3Y
vidcount=df_f3Y.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f3Y")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f3Y"]=df_channel["videoCount_f3Y"].fillna(value=0)#set NAs into 0
#f4Y
vidcount=df_f4Y.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f4Y")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f4Y"]=df_channel["videoCount_f4Y"].fillna(value=0)#set NAs into 0
#f5Y
vidcount=df_f5Y.channelId.value_counts().rename_axis("channelId").reset_index(name="videoCount_f5Y")#video counts for the period in new df
df_channel=pd.merge(df_channel, vidcount, on="channelId", how="left")#merge into df_channel
df_channel["videoCount_f5Y"]=df_channel["videoCount_f5Y"].fillna(value=0)#set NAs into 0

In [22]:
#extract commentCount, like, dislike, view for latest periods

#f1W
df_f1W=df_f1W.rename(columns={"viewCount":"viewCount_f1W", "likeCount":"likeCount_f1W", "dislikeCount":"dislikeCount_f1W" ,"commentCount":"commentCount_f1W"})
like=df_f1W.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f1M
df_f1M=df_f1M.rename(columns={"viewCount":"viewCount_f1M", "likeCount":"likeCount_f1M", "dislikeCount":"dislikeCount_f1M" ,"commentCount":"commentCount_f1M"})
like=df_f1M.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f3M
df_f3M=df_f3M.rename(columns={"viewCount":"viewCount_f3M", "likeCount":"likeCount_f3M", "dislikeCount":"dislikeCount_f3M" ,"commentCount":"commentCount_f3M"})
like=df_f3M.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f6M
df_f6M=df_f6M.rename(columns={"viewCount":"viewCount_f6M", "likeCount":"likeCount_f6M", "dislikeCount":"dislikeCount_f6M" ,"commentCount":"commentCount_f6M"})
like=df_f6M.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f1Y
df_f1Y=df_f1Y.rename(columns={"viewCount":"viewCount_f1Y", "likeCount":"likeCount_f1Y", "dislikeCount":"dislikeCount_f1Y" ,"commentCount":"commentCount_f1Y"})
like=df_f1Y.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f2Y
df_f2Y=df_f2Y.rename(columns={"viewCount":"viewCount_f2Y", "likeCount":"likeCount_f2Y", "dislikeCount":"dislikeCount_f2Y" ,"commentCount":"commentCount_f2Y"})
like=df_f2Y.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f3Y
df_f3Y=df_f3Y.rename(columns={"viewCount":"viewCount_f3Y", "likeCount":"likeCount_f3Y", "dislikeCount":"dislikeCount_f3Y" ,"commentCount":"commentCount_f3Y"})
like=df_f3Y.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f4Y
df_f4Y=df_f4Y.rename(columns={"viewCount":"viewCount_f4Y", "likeCount":"likeCount_f4Y", "dislikeCount":"dislikeCount_f4Y" ,"commentCount":"commentCount_f4Y"})
like=df_f4Y.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel
#f5Y
df_f5Y=df_f5Y.rename(columns={"viewCount":"viewCount_f5Y", "likeCount":"likeCount_f5Y", "dislikeCount":"dislikeCount_f5Y" ,"commentCount":"commentCount_f5Y"})
like=df_f5Y.groupby(["channelId"]).sum()#get sum
like=like.reset_index().fillna(value=0)
df_channel=pd.merge(df_channel, like, on="channelId", how="left")#merge like and channel


In [23]:
df_channel.to_csv("AllData.csv")